In [4]:
import json
import math
import random
import time
from math import *

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, make_circles
from tabulate import tabulate

from kdtreen import *
from utils import *

In [5]:
no_points = 100
no_centres = 5

In [6]:
X, Y = make_blobs(n_samples=no_points, centers=no_centres)
points = list(set([(x, y) for x, y in X]))
# points = list(set([(round(x, 1), round(y, 1)) for x, y in X]))
print(len(points))

100


In [7]:
import math


def calculate_distance(point1, point2):
    return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)


def create_distance_matrix(points):
    n = len(points)
    distance_matrix = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            distance = calculate_distance(points[i], points[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix


def primMST(graph):
    num_vertices = len(graph)
    key = [float("inf")] * num_vertices
    parent = [-1] * num_vertices
    mstSet = [False] * num_vertices
    key[0] = 0
    for cout in range(num_vertices):
        u = minKey(key, mstSet)
        mstSet[u] = True
        for v in range(num_vertices):
            if 0 < graph[u][v] < key[v] and not mstSet[v]:
                key[v] = graph[u][v]
                parent[v] = u
    return parent


def minKey(key, mstSet):
    min_val = float("inf")
    min_index = -1
    for v in range(len(key)):
        if key[v] < min_val and not mstSet[v]:
            min_val, min_index = key[v], v
    return min_index


def get_MST_edges_with_coordinates(points):
    distance_matrix = create_distance_matrix(points)
    mst_parent = primMST(distance_matrix)
    edges = []
    for i in range(1, len(points)):
        if mst_parent[i] != -1:  # Ensure there's a parent
            edge = (points[mst_parent[i]], points[i])
            edges.append(edge)
    return edges


In [8]:
savefile = "mst_datatest.json"
to_plot = False
no_points = 10000
no_centres = 2

In [9]:
if no_points > 999:
    to_plot = False

In [10]:
X, Y = make_blobs(n_samples=no_points, centers=no_centres)
points = list(set([(x, y) for x, y in X]))
# points = list(set([(round(x, 1), round(y, 1)) for x, y in X]))
print(len(points))

10000


In [11]:
dcran_start_time = time.time()
i_neighbors_dict = {}
tree = KDTree()
G = nx.Graph()

In [12]:
def build(points):
    global cordmap, i_neighbors_dict, tree, G
    tree.root = tree.build(points)
    limit_dist = math.ceil(log(len(points)))

    G.add_nodes_from(points)
    for point in points:
        i_neighbors_dict[point] = i_neighbors(tree, point, limit_dist)
    print("max dis : " ,limit_dist )
    return tree, G

In [13]:
def dcran(points):
    global cordmap, i_neighbors_dict, tree, G
    tree, G = build(points)

    for k in range(math.ceil(log(len(points)))):
        components = list(nx.connected_components(G))
        if len(components) == 1:
            break
        for component in components:
            for pointi in component:
                eudis, pointj = i_neighbors_dict[pointi][k]
                if pointj in component:
                    continue

                G.add_edge(pointi, pointj, weight=eudis)
    print(len(list(nx.connected_components(G))))

In [14]:
dcran(points)

max dis :  10
1


In [15]:
import numpy as np

centroids = []
components = list(nx.connected_components(G))

for component in components:
    points = np.array(list(component))
    centroid = points.mean(axis=0)
    centroids.append(centroid)
    print(f"The centroid of the component is {centroid}")

The centroid of the component is [-7.0429783  -0.02059707]


In [16]:
from scipy.spatial import distance

# Assuming centroids is already defined as shown in previous steps
centroids_x = sorted(enumerate(centroids), key=lambda x: x[1][0])
centroids_y = sorted(enumerate(centroids), key=lambda x: x[1][1])

tot_dis_x = 0
tot_dis_y = 0

for i in range(len(centroids_x) - 1):
    tot_dis_x += distance.euclidean(centroids_x[i][1], centroids_x[i + 1][1])
    tot_dis_y += distance.euclidean(centroids_y[i][1], centroids_y[i + 1][1])

if tot_dis_x > tot_dis_y:
    sorted_indices = [index for index, _ in centroids_y]
else:
    sorted_indices = [index for index, _ in centroids_x]

# sorted_indices will contain the indices of centroids, ordered by the selected sorting criterion

In [17]:
from scipy.spatial import distance

def get_closest_points(c1, c2):
    points = np.array(list(c1))
    centroid = points.mean(axis=0)

    # Compute the distances between the centroid and each point in c2
    distances = distance.cdist([centroid], c2)

    # Get the index of the closest point
    p2 = c2[np.argmin(distances)]
    distances_to_c1 = distance.cdist([p2], c1)[0]
    p1 = c1[np.argmin(distances_to_c1)]
    # Return the closest point
    return (p1 , p2)

In [18]:

for i in range(len(sorted_indices) - 1):
    # Get the components corresponding to the current and next centroid in the sorted list
    component_a = components[sorted_indices[i]]
    component_b = components[sorted_indices[i + 1]]

    # Find the closest points between component_a and component_b
    point_a, point_b = get_closest_points(component_a, component_b)

    # Assuming a function add_edge(point_a, point_b) that adds an edge between two points
    G.add_edge(point_a, point_b)

    print(
        f"Connected component {sorted_indices[i]} to component {sorted_indices[i+1]} through points {point_a} and {point_b}"
    )

In [19]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")

In [20]:
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time
print(dcran_elapsed_time)

1.1241521835327148


In [21]:
stmst_start_time = time.time()

In [22]:



def calculate_distance(point1, point2):
    return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)


def create_distance_matrix(points):
    n = len(points)
    distance_matrix = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            distance = calculate_distance(points[i], points[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix


def primMST(graph):
    num_vertices = len(graph)
    key = [float("inf")] * num_vertices
    parent = [-1] * num_vertices
    mstSet = [False] * num_vertices
    key[0] = 0

    for cout in range(num_vertices):
        u = minKey(key, mstSet)
        mstSet[u] = True
        for v in range(num_vertices):
            if 0 < graph[u][v] < key[v] and mstSet[v] is False:
                key[v] = graph[u][v]
                parent[v] = u
    return parent


def minKey(key, mstSet):
    min = float("inf")
    min_index = -1
    for v in range(len(key)):
        if key[v] < min and not mstSet[v]:
            min, min_index = key[v], v
    return min_index


# Example list of points

# Create the distance matrix
distance_matrix = create_distance_matrix(points)

# Generate MST using Prim's algorithm
mst = primMST(distance_matrix)

# Print the edges in the MST


In [23]:
stmst_end_time = time.time()
stmst_elapsed_time = stmst_end_time - stmst_start_time
print(stmst_elapsed_time)   

75.49475121498108


In [24]:
stmst_elapsed_time/dcran_elapsed_time

67.15705606489537